In [1]:
import numpy as np

In [2]:
def random_walk(s0, mu, sigma, days):
    dt = 1/365.
    prices = np.zeros(days)
    shocks = np.zeros(days)
    prices[0] = s0
    for i in range(1, days):
        e = np.random.normal(loc=mu * dt, scale=sigma * np.sqrt(dt))
        prices[i] = prices[i-1] * (1 + e)
    return prices

In [3]:
days = 365 * 4  # days to expiry
s0 = 100  # current underlying price
mu = 0.02  # drift
sigma = 0.2  # volatility
K = 100  # strike price

In [4]:
A = np.average(random_walk(s0, mu, sigma, days))

In [5]:
C = max(0, A - K)

In [6]:
n = 10000

In [7]:
%time np.average([max(0, np.average(random_walk(s0, mu, sigma, days)) - K) for i in range(0, n)])

CPU times: user 2min 23s, sys: 59.5 ms, total: 2min 23s
Wall time: 2min 23s


11.630020852770665

In [16]:
from dask.distributed import Client

In [9]:
tritonCluster = Client("tcp://192.168.86.42:8786")

In [10]:
tritonCluster

Client Scheduler: tcp://192.168.86.42:8786 Dashboard: http://192.168.86.42:8787/status,Cluster Workers: 4 Cores: 16 Memory: 7.75 GB


In [11]:
from dask import delayed

In [12]:
s0 = 100
K = 100
mu = 0.02
sigma = 0.2
days = 365*4
n = 10000

In [13]:
result = delayed(np.average)([
    delayed(max)(
        0, 
        delayed(np.average)(random_walk(s0, mu, sigma, days)) - K
    ) for i in range(0, n)
])

In [19]:
%time result.compute()

CPU times: user 2.23 s, sys: 184 ms, total: 2.42 s
Wall time: 35.8 s


11.819844982493876